In [187]:
import pandas as pd
import gcsfs
import json

import os
import pprint as pp

import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring
from google.cloud.aiplatform.explain.metadata.tf.v2 import \
    saved_model_metadata_builder

In [188]:
fs = gcsfs.GCSFileSystem()

with fs.open("mlops_rasa_bucket/dataset/features.txt", "r") as f:
    features = [line.split()[1] for line in f.readlines()]
# features = pd.read_csv("gs://mlops_rasa_bucket/dataset/features.txt", sep="\s+", header=None,names=["index", "feature_name"])
# features.drop(columns=['index'], inplace=True)
# features = features["feature_name"].tolist()
# print(features)
label_map = {1: 'WALKING', 2:'WALKING_UPSTAIRS',3:'WALKING_DOWNSTAIRS',\
                       4:'SITTING', 5:'STANDING',6:'LAYING'}

In [189]:
import pandas as pd
from collections import Counter

# Assuming you've already read these:
# X_train = pd.read_csv(..., sep=r"\s+", header=None)
# X_test  = pd.read_csv(..., sep=r"\s+", header=None)
# features = pd.read_csv(..., sep=r"\s+", header=None, names=["idx", "name"])

feature_names = features

# 1. Count all names
name_counts = Counter(feature_names)

# 2. Prepare a new names list we will mutate
new_names = feature_names.copy()

# 3. For every name with count > 1, rename its occurrences to name_1, name_2, ...
for name, cnt in name_counts.items():
    if cnt > 1:
        # get all indices where this name appears
        idxs = [i for i, n in enumerate(feature_names) if n == name]
        # rename them as name_1, name_2, ...
        for j, idx in enumerate(idxs, start=1):
            new_names[idx] = f"{name}_{j}"
print(len(set(new_names)))
# Optional sanity check: no duplicates now
from collections import Counter as C2
dup_check = [n for n, c in C2(new_names).items() if c > 1]
print("Duplicates after renaming:", dup_check)  # should be []
# print(Counter(new_names))
# # 4. Assign to both train and test
# X_train.columns = new_names
# X_test.columns = new_names


561
Duplicates after renaming: []


In [190]:
X_train = pd.read_csv("gs://mlops_rasa_bucket/dataset/X_train.txt", sep="\s+", header=None)
X_train.columns = new_names

y_train = pd.read_csv("gs://mlops_rasa_bucket/dataset/y_train.txt", sep="\s+", header=None)
y_train.columns = ["activity"]

X_test = pd.read_csv("gs://mlops_rasa_bucket/dataset/X_test.txt", sep="\s+", header=None)
X_test.columns = new_names

y_test = pd.read_csv("gs://mlops_rasa_bucket/dataset/y_test.txt", sep="\s+", header=None)
y_test.columns = ["activity"]

X_train.head()


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [191]:
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

X_train:  (7352, 561)
y_train:  (7352, 1)
X_test:  (2947, 561)
y_test:  (2947, 1)


In [192]:
import numpy as np

# Ensure X/y are plain numpy arrays
X_train_np = np.asarray(X_train)
y_train_np = np.asarray(y_train)
# print(y_train_np.count())

X_test_np = np.asarray(X_test)
y_test_np = np.asarray(y_test)


In [326]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

def print_metrics(name, y_true, y_pred):
    print(f"\n=== {name} ===")
    print("Acuracy:", accuracy_score(y_true, y_pred))
    print("F1:", f1_score(y_true, y_pred, average="micro"))
    #r2 = r2_score(y_true, y_pred)
    #print(f"R2   : {r2:.4f}")

In [327]:
from flaml import AutoML
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_np)
automl = AutoML()
automl.fit(
    X_train_scaled, y_train_np,
    task="classification",
    metric="macro_f1",
    time_budget=60,
    #average='weighted',
    # estimator_list = ['rf', 'extra_tree', 'xgb_limitdepth', 'sgd', 'lrl1'],
    seed=42
)

best_model = automl.model.estimator


[flaml.automl.logger: 12-11 18:36:14] {1752} INFO - task = classification
[flaml.automl.logger: 12-11 18:36:14] {1763} INFO - Evaluation method: holdout
[flaml.automl.logger: 12-11 18:36:14] {1862} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 12-11 18:36:14] {1979} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'lrl1']
[flaml.automl.logger: 12-11 18:36:14] {2282} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 12-11 18:36:15] {2417} INFO - Estimated sufficient time budget=12186s. Estimated necessary time budget=282s.
[flaml.automl.logger: 12-11 18:36:15] {2466} INFO -  at 1.4s,	estimator lgbm's best error=0.0786,	best estimator lgbm's best error=0.0786
[flaml.automl.logger: 12-11 18:36:15] {2282} INFO - iteration 1, current learner lgbm


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names



[flaml.automl.logger: 12-11 18:36:16] {2466} INFO -  at 2.5s,	estimator lgbm's best error=0.0786,	best estimator lgbm's best error=0.0786
[flaml.automl.logger: 12-11 18:36:16] {2282} INFO - iteration 2, current learner lgbm


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names



[flaml.automl.logger: 12-11 18:36:17] {2466} INFO -  at 3.5s,	estimator lgbm's best error=0.0556,	best estimator lgbm's best error=0.0556
[flaml.automl.logger: 12-11 18:36:17] {2282} INFO - iteration 3, current learner sgd


...

[flaml.automl.logger: 12-11 18:36:47] {2466} INFO -  at 33.1s,	estimator xgboost's best error=0.0470,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:36:47] {2282} INFO - iteration 21, current learner rf
[flaml.automl.logger: 12-11 18:36:47] {2466} INFO -  at 33.6s,	estimator rf's best error=0.1506,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:36:47] {2282} INFO - iteration 22, current learner rf
[flaml.automl.logger: 12-11 18:36:48] {2466} INFO -  at 34.4s,	estimator rf's best error=0.0879,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:36:48] {2282} INFO - iteration 23, current learner lgbm
[flaml.automl.logger: 12-11 18:36:50] {2466} INFO -  at 36.5s,	estimator lgbm's best error=0.0000,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:36:50] {2282} INFO - iteration 24, current learner rf


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names



[flaml.automl.logger: 12-11 18:36:50] {2466} INFO -  at 36.8s,	estimator rf's best error=0.0879,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:36:50] {2282} INFO - iteration 25, current learner extra_tree
[flaml.automl.logger: 12-11 18:36:51] {2466} INFO -  at 37.0s,	estimator extra_tree's best error=0.1805,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:36:51] {2282} INFO - iteration 26, current learner sgd
[flaml.automl.logger: 12-11 18:37:01] {2466} INFO -  at 47.8s,	estimator sgd's best error=0.3654,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:37:01] {2282} INFO - iteration 27, current learner rf
[flaml.automl.logger: 12-11 18:37:03] {2466} INFO -  at 49.5s,	estimator rf's best error=0.0879,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:37:03] {2282} INFO - iteration 28, current learner xgboost
[flaml.automl.logger: 12-11 18:37:08] {2466} INFO -  at 54.2s,	estimator xgboost's best e

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names



[flaml.automl.logger: 12-11 18:37:12] {2466} INFO -  at 58.8s,	estimator rf's best error=0.0879,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:37:12] {2282} INFO - iteration 33, current learner rf
[flaml.automl.logger: 12-11 18:37:13] {2466} INFO -  at 59.1s,	estimator rf's best error=0.0879,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:37:13] {2282} INFO - iteration 34, current learner extra_tree
[flaml.automl.logger: 12-11 18:37:13] {2466} INFO -  at 59.4s,	estimator extra_tree's best error=0.1432,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:37:13] {2282} INFO - iteration 35, current learner extra_tree
[flaml.automl.logger: 12-11 18:37:13] {2466} INFO -  at 59.6s,	estimator extra_tree's best error=0.1432,	best estimator lgbm's best error=0.0000
[flaml.automl.logger: 12-11 18:37:13] {2282} INFO - iteration 36, current learner extra_tree
[flaml.automl.logger: 12-11 18:37:13] {2466} INFO -  at 59.7s,	estimator

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# y_train_en = le.fit_transform(y_train['activity'])
# y_test_en = le.fit_transform(y_test['activity'])

y_train_en = y_train_np
y_test_en = y_test_np

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", best_model),
])

pipeline.fit(X_train_np, y_train_en)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

pred = pipeline.predict(X_test_np)
print("ACC:", accuracy_score(y_test_en, pred))
print("F1:", f1_score(y_test_en, pred, average="macro"))


In [ ]:
y_train_en = y_train_np
y_test_en = y_test_np

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

pipeline = Pipeline([
    ("model", RandomForestClassifier(
    n_estimators=150,
    max_depth=15,
    min_samples_leaf=10,
    min_samples_split=20,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    oob_score=True,
)),
])

pipeline.fit(X_train, y_train)


y_test_pred  = pipeline.predict(X_test)

print_metrics('Test', y_test, y_test_pred)


In [ ]:
import numpy as np
import pandas as pd

importances = rf.feature_importances_          # shape (n_features,)

feat_imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importances,
})

feat_imp = feat_imp.sort_values(
    "importance", ascending=False
).reset_index(drop=True)

print(feat_imp.head(20))    # top 20 features


In [ ]:
import joblib
joblib.dump(rf_pipeline, "model.joblib")


In [322]:
!gsutil cp model.joblib gs://mlops_rasa_bucket/models/


Copying file://model.joblib [Content-Type=application/octet-stream]...
- [1 files][ 11.4 MiB/ 11.4 MiB]                                                
Operation completed over 1 objects/11.4 MiB.                                     


In [323]:
from google.cloud import aiplatform

aiplatform.init(
    project="mlops-rasa",
    location="us-central1"
)

model = aiplatform.Model.upload(
    display_name="RF_pipeline",
    artifact_uri="gs://mlops_rasa_bucket/models/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-4:latest",
    # explanation_metadata="gs://mlops_rasa_bucket/schema/schema.json"
)

endpoint = model.deploy(
    machine_type="n1-standard-2",
    min_replica_count=1,
    max_replica_count=1,
    enable_access_logging=True, 
)



Creating Model
Create Model backing LRO: projects/538975033228/locations/us-central1/models/399033660440838144/operations/1152383482643611648
Model created. Resource name: projects/538975033228/locations/us-central1/models/399033660440838144@1
To use this Model in another session:
model = aiplatform.Model('projects/538975033228/locations/us-central1/models/399033660440838144@1')
Creating Endpoint
Create Endpoint backing LRO: projects/538975033228/locations/us-central1/endpoints/3300495312560324608/operations/9153872645597429760
Endpoint created. Resource name: projects/538975033228/locations/us-central1/endpoints/3300495312560324608
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/538975033228/locations/us-central1/endpoints/3300495312560324608')
Deploying model to Endpoint : projects/538975033228/locations/us-central1/endpoints/3300495312560324608
Deploy Endpoint model backing LRO: projects/538975033228/locations/us-central1/endpoints/33004953125603246

KeyboardInterrupt: 

In [ ]:
instances_all = X_test.to_numpy().tolist()

preds = endpoint.predict(instances=instances_all[:5])
print(preds.predictions)
print(y_test_np[:5])


In [ ]:
instances_train_all = X_train.to_numpy().tolist()
y_preds_train = []
for i in range(0, len(y_train_np), 100):
    preds = endpoint.predict(instances=instances_train_all[i:i+100])
    y_preds_train.extend(preds.predictions)
    # print(y_test_en[:5])
    

In [ ]:
y_preds = []
for i in range(0, len(y_test_np), 100):
    preds = endpoint.predict(instances=instances_all[i:i+100])
    y_preds.extend(preds.predictions)
    # print(y_test_en[:5])
    

In [ ]:
print_metrics('Train', y_train_np, y_preds_train)
print_metrics('Test', y_test_np, y_preds)

In [ ]:
from evidently import Report, DataDefinition, Dataset, MulticlassClassification
from evidently.presets import DataDriftPreset, RegressionPreset, ClassificationPreset

test_base = X_test.copy()
test_base['activity'] = y_test_en
test_base["prediction"] = y_preds

train_monitor = X_train.copy()
train_monitor['activity'] = y_train_en
train_monitor["prediction"] = y_preds_train

schema = DataDefinition(
    numerical_columns=new_names,       
    classification=[MulticlassClassification(
        target="activity",
        prediction_labels="prediction",
        # labels={0.0: 'WALKING', 1.0:'WALKING_UPSTAIRS',2.0:'WALKING_DOWNSTAIRS',
        #                3:'SITTING', 4:'STANDING',5:'LAYING'}
    )],
)


In [ ]:
from evidently.metrics import *
def run_evidently_report(reference_df, current_df, html_path, label="", schema=schema):
    """
    Wraps Evidently report creation using the new Dataset/DataDefinition API.
    """

    ref_dataset = Dataset.from_pandas(
        reference_df,
        data_definition=schema,
    )
    cur_dataset = Dataset.from_pandas(
        current_df,
        data_definition=schema,
    )

    report = Report([
        DataDriftPreset(),
        # Accuracy(),
        # Precision(),
        # Recall()
    ])

    # Evidently >=0.4: run(current, reference)
    snapshot = report.run(cur_dataset, ref_dataset)
    snapshot.save_html(html_path)
    print(f"{label}Evidently report saved to: {html_path}")


In [ ]:
# Base train vs test monitoring (optional but nice)
run_evidently_report(
    reference_df=train_monitor,
    current_df=test_base,
    html_path="evidently_train_vs_test.html",
    label="[Base train vs test] ",
    schema=schema
)


In [ ]:
X_test_changed = X_test.copy()
X_test_changed['tBodyAcc-mean()-X'] = np.random.randint(0, 101, size=len(X_test_changed))
X_test_changed['tBodyAcc-mean()-Y'] = np.random.randint(0, 101, size=len(X_test_changed))
X_test_changed['tBodyAcc-mean()-Z'] = np.random.randint(0, 101, size=len(X_test_changed))

instances_changed = X_test_changed.to_numpy().tolist()

y_preds_rand = []
for i in range(0, len(y_test_en), 100):
    preds = endpoint.predict(instances=instances_changed[i:i+100])
    y_preds_rand.extend(preds.predictions)
print_metrics('Test', y_test_en, y_preds_rand)

test_base_rand = X_test_changed.copy()
test_base_rand['activity'] = y_test_en
test_base_rand["prediction"] = y_preds_rand
                                  
'''run_evidently_report(
    reference_df=test_base,
    current_df=test_base_rand,
    html_path="tBodyAcc-mean()-X.html",
    label="tBodyAcc-mean()-X ",
    schema=schema
)
'''

In [ ]:
col_a = "fBodyAccJerk-mean()-X"
col_b = "fBodyAcc-mean()-X"

# 1. Create a modified copy of X_test with swapped columns
X_test_swapped = X_test.copy()

# Actually swap values
tmp = X_test_swapped[col_a].copy()
X_test_swapped[col_a] = X_test_swapped[col_b]
X_test_swapped[col_b] = tmp

# 2. Convert to list of instances for Vertex
instances_swapped = X_test_swapped.to_numpy().tolist()

# 3. Get predictions from deployed endpoint
y_preds_swapped = []
for i in range(0, len(X_test_swapped), 100):
    batch = instances_swapped[i:i+100]
    preds = endpoint.predict(instances=batch)
    y_preds_swapped.extend(preds.predictions)

# 4. Print metrics (your helper)
print_metrics("Swapped Columns", y_test_en, y_preds_swapped)

# 5. Build monitoring dataframe
test_base_swapped = X_test_swapped.copy()
test_base_swapped["activity"] = y_test_en
test_base_swapped["prediction"] = y_preds_swapped

# 6. Run Evidently report with fresh schema
'''run_evidently_report(
    reference_df=test_base,
    current_df=test_base_swapped,
    html_path="swap_cols_report_BodyAcc.html",
    label="Column Swap Drift"
)
'''

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# if your model is in a pipeline:
rf = pipeline.named_steps["model"]

importances = rf.feature_importances_
feature_names = np.array(X_train.columns)

feat_imp = pd.DataFrame({
    "feature": feature_names,
    "importance": importances,
}).sort_values("importance", ascending=False)

N = 20
top_feats = feat_imp.head(N)

fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(range(len(top_feats)), top_feats["importance"][::-1])
ax.set_yticks(range(len(top_feats)))
ax.set_yticklabels(top_feats["feature"][::-1])
ax.set_xlabel("Feature importance")
ax.set_title("Random Forest Feature Importances (Top 10)")
fig.tight_layout()

fig.savefig("rf_feature_importance.png", dpi=150)
print("Saved to rf_feature_importance.png")


In [ ]:
X_test_changed = X_test.copy()

gravity_cols = [c for c in X_test.columns if "GravityAcc" in c or "gravityMean" in c]
X_test_changed[gravity_cols] = np.random.randn(len(X_test), len(gravity_cols))

# X_test_changed.iloc[0] *= 50
# X_test_changed.iloc[1] += 0.7

instances_changed = X_test_changed.to_numpy().tolist()

y_preds_rand = []
for i in range(0, len(y_test_en), 100):
    preds = endpoint.predict(instances=instances_changed[i:i+100])
    y_preds_rand.extend(preds.predictions)
print_metrics('Test', y_test_en, y_preds_rand)

test_base_rand = X_test_changed.copy()
test_base_rand['activity'] = y_test_en
test_base_rand["prediction"] = y_preds_rand
                                  
'''run_evidently_report(
    reference_df=test_base,
    current_df=test_base_rand,
    html_path="Gravity.html",
    label="Gravity.html ",
    schema=schema
)
'''

In [ ]:
topk = feat_imp["feature"].head(10).tolist()   # list of 10 feature names
X_test_changed = X_test.copy()

# rotate: f1->f2, f2->f3, ..., f10->f1
shifted = topk[1:] + [topk[0]]

X_test_changed[topk] = X_test_changed[shifted].to_numpy()

instances_changed = X_test_changed.to_numpy().tolist()

y_preds_rand = []
for i in range(0, len(y_test_en), 100):
    preds = endpoint.predict(instances=instances_changed[i:i+100])
    y_preds_rand.extend(preds.predictions)
print_metrics('Test', y_test_en, y_preds_rand)

test_base_rand = X_test_changed.copy()
test_base_rand['activity'] = y_test_en
test_base_rand["prediction"] = y_preds_rand
                                  
'''run_evidently_report(
    reference_df=test_base,
    current_df=test_base_rand,
    html_path="top_10_swap.html",
    label="top_10_swap",
    schema=schema
)
'''
